NOW WE TRAIN AND VALIDATE ON ALL (ITEM, SHOP) EVER SEEN IN DATA
Shold be:
train - all (item ,shop) previosly seen
test - dot product of (item, shop) in current dbn

Now it is as it should be

In [1]:
FIRST_N_MONTH_TO_DROP = 6

In [2]:
import numpy as np
import pandas as pd

import seaborn as sns
from sklearn.preprocessing import TargetEncoder
import gc

import numba

import matplotlib.pyplot as plt


In [3]:
#!pip install torch

In [4]:
data_train = pd.read_csv('../data_cleaned/data_train.csv')
item_categories = pd.read_csv('../data_cleaned/item_categories.csv')
items = pd.read_csv('../data_cleaned/items.csv')
shops = pd.read_csv('../data_cleaned/shops.csv')
test = pd.read_csv('../data_cleaned/test.csv')
test['date_block_num'] = 34
data_train = data_train.merge(test, on=['item_id','shop_id','date_block_num'], how='outer').fillna(0)

In [5]:


data_train=data_train.reset_index()
data_train = data_train.sort_values(by='date_block_num')
train = data_train.copy()


In [6]:
def create_change(table):
    table=table.copy().fillna(0)
    #cols = [col for col in table.columns if isinstance(col, int)]
    cols = [col for col in table.columns if col.isdigit()]
    #print(cols)
    cols.sort()
    result = table.copy()
    for col in cols[1:]:
    
        result[col] = (table[col] / table[str(int(col)-1)] - 1) * 100
        
    result = result.replace([np.inf], 0)
    result = result.replace([np.nan], 0)
    return result
        
    

In [7]:
import time
def create_lag_table_general(data,column='item_cnt_monthitem_id_lag_1',
                             whole=None,
                             groupby = ['item_id'],
                             item_shop_city_category_sup_category=None):#long - ~467s
    #whole contains lagged, which means for dbn = 24 lag_1 calculated on 23
    #(item cnt month for dbn==24 stored in lag_1 for dbn == 25 )
    #to make whole same format as data, we need to whole['date_block_num']-=1
    start = time.time()

    whole=whole.copy()
    new_table = data.copy()
    if 'lag' in column:
        whole['date_block_num'] -= 1
        whole = whole[whole['date_block_num'] >= 0]

    
    groped_item = whole.groupby(['date_block_num',*groupby])[column].unique()
    
    print(groped_item.map(lambda a:a[0]).describe())

    print(whole.date_block_num)
    agg = 'mean'
    df = pd.pivot_table(whole, 
                    index=groupby,
                    columns=['date_block_num'],
                    values=[column],
                    aggfunc=agg, 
                    fill_value=0
                   ).reset_index()

    train = pd.DataFrame({col:df[col] for col in groupby})
    
    col = df[column]
    
    train = pd.concat([train,col], axis=1)

    numerical = [col for col in new_table.columns if isinstance(col, int)]
    numerical = [col for col in numerical if col < 34]
    new_table = item_shop_city_category_sup_category.merge(train, how='left').fillna(0)
    new_table = new_table[['shop_id','item_id', *numerical]]
    print(new_table)
    return new_table
        

In [8]:
def create_item_shop_data(data, column='item_cnt_day'):
    if column == 'item_cnt_day':
        agg = 'sum'
    else:
        agg='mean'
    df = pd.pivot_table(data, 
                    index=['shop_id','item_id'],
                    columns=['date_block_num'],
                    values=[column],
                    aggfunc=agg, 
                    fill_value=0
                   ).reset_index()
   # print(df)
    train = pd.DataFrame({'shop_id':df['shop_id'], 'item_id':df['item_id']})
    
    item_cnt_day = df[column]
    train = pd.concat([train,item_cnt_day], axis=1)
    #print(items)
    merged = train.merge(items, how='left').merge(item_categories, how='left').merge(shops, how='left') \
        .drop(['item_name','item_category_name','shop_name'], axis=1)

    merged.columns = [str(col) for col in merged.columns]
    return merged

In [9]:
whole = pd.read_csv('../data.csv')

In [10]:
shop_city = whole.groupby('shop_id')['city'].unique().reset_index()

In [11]:
category_super_category = whole.groupby('item_category_id')['super_category'].unique().reset_index()

In [12]:

merged = create_item_shop_data(data_train)

In [13]:
merged.columns

Index(['shop_id', 'item_id', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21',
       '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33',
       '34', 'item_category_id'],
      dtype='object')

In [14]:
def write_file_by_name(merged,column=None, whole=None, groupby=''):
    time1 = time.time()
    item_shop_city_category_sup_category = merged.merge(shop_city). \
    merge(category_super_category)[['shop_id',	'item_id','item_category_id','city','super_category']]
    
    item_shop_city_category_sup_category['city'] = item_shop_city_category_sup_category['city'].map(lambda a:a[0])
    item_shop_city_category_sup_category['super_category'] = item_shop_city_category_sup_category['super_category'].map(lambda a:a[0])

    table_ema6_item = create_lag_table_general(merged,
                                               column=column,
                                               whole=whole,
                                               groupby=groupby, 
                                               item_shop_city_category_sup_category=item_shop_city_category_sup_category)
    
    table_ema6_item.to_csv(f'data/{column}.csv', index=False)
    del table_ema6_item
    gc.collect()

    print('time taken for',column,':', time.time() - time1  )

In [15]:
whole.columns

Index(['Unnamed: 0', 'shop_id', 'item_id', 'date_block_num', 'avg_item_price',
       'item_cnt_month', 'item_name', 'item_category_id', 'item_category_name',
       'shop_name', 'month', 'year', 'super_category', 'city', 'shop_id_cat',
       'item_id_cat', 'item_category_id_cat', 'ema_6_item_cnt_month_item_id',
       'ema_6_item_cnt_month_item_id_city',
       'ema_6_item_cnt_month_item_category_id_cat_shop_id',
       'ema_6_item_cnt_month_item_category_id_cat_city',
       'ema_6_item_cnt_month_item_id_shop_id', 'item_cnt_month_lag_1',
       'item_cnt_month_lag_2', 'item_cnt_month_lag_3', 'item_cnt_month_lag_4',
       'item_cnt_month_lag_5', 'item_cnt_month_lag_6', 'avg_item_price_lag_1',
       'item_cnt_monthitem_id_lag_1', 'item_cnt_monthitem_id_lag_2',
       'item_cnt_monthitem_id_lag_3', 'item_cnt_monthitem_id_lag_4',
       'item_cnt_monthitem_id_lag_5', 'item_cnt_monthitem_id_lag_6',
       'avg_item_priceitem_id_lag_1', 'item_cnt_monthshop_id_lag_1',
       'item_cnt_mo

In [15]:
"""
write_file_by_name(merged,column='ema_6_item_cnt_month_item_id', whole=whole, groupby=['item_id'])
#write_file_by_name(merged,column='ema_6_item_cnt_month_item_id_shop_id', whole=whole, groupby=['item_id','shop_id'])
write_file_by_name(merged,column='ema_6_item_cnt_month_item_category_id_cat_city', whole=whole, groupby=['item_category_id','city'])
write_file_by_name(merged,column='ema_6_item_cnt_month_item_category_id_cat_shop_id', whole=whole, groupby=['item_category_id','shop_id'])

write_file_by_name(merged,column='date_block_num_diff', whole=whole, groupby=['item_id'])
write_file_by_name(merged,column='avg_item_priceitem_id_lag_1', whole=whole, groupby=['item_id'])
write_file_by_name(merged,column='item_cnt_monthitem_category_id_cat_lag_1', whole=whole, groupby=['item_category_id'])
write_file_by_name(merged,column='avg_item_price', whole=whole, groupby=['item_id','shop_id'])
write_file_by_name(merged,column='item_cnt_month_lag_1', whole=whole, groupby=['item_id'])
"""

"\nwrite_file_by_name(merged,column='ema_6_item_cnt_month_item_id', whole=whole, groupby=['item_id'])\n#write_file_by_name(merged,column='ema_6_item_cnt_month_item_id_shop_id', whole=whole, groupby=['item_id','shop_id'])\nwrite_file_by_name(merged,column='ema_6_item_cnt_month_item_category_id_cat_city', whole=whole, groupby=['item_category_id','city'])\nwrite_file_by_name(merged,column='ema_6_item_cnt_month_item_category_id_cat_shop_id', whole=whole, groupby=['item_category_id','shop_id'])\n\nwrite_file_by_name(merged,column='date_block_num_diff', whole=whole, groupby=['item_id'])\nwrite_file_by_name(merged,column='avg_item_priceitem_id_lag_1', whole=whole, groupby=['item_id'])\nwrite_file_by_name(merged,column='item_cnt_monthitem_category_id_cat_lag_1', whole=whole, groupby=['item_category_id'])\nwrite_file_by_name(merged,column='avg_item_price', whole=whole, groupby=['item_id','shop_id'])\nwrite_file_by_name(merged,column='item_cnt_month_lag_1', whole=whole, groupby=['item_id'])\n"

In [16]:
#pr[pr['0'] != 0]['0'].describe()
#whole[whole['date_block_num'] == 0][['shop_id', 'item_id','avg_item_price']].describe()

In [17]:
"""
table_item_price = pd.read_csv('data/avg_item_priceitem_id_lag_1.csv')
price_changes_items = create_change(table_item_price)
price_changes_items.to_csv('data/item_price_change.csv', index=False)
del table_item_price
del price_changes_items
gc.collect()

prices = pd.read_csv('data/avg_item_price.csv')
price_changes = create_change(prices)
price_changes.to_csv('data/shop_item_price_change.csv', index=False)
del price_changes
del prices
gc.collect()

cnt_changes_items_shops = create_change(merged)
cnt_changes_items_shops.to_csv('data/cnt_changes_items_shops.csv', index=False)
del cnt_changes_items_shops
gc.collect()


table_item_cnt = pd.read_csv('data/item_cnt_month_lag_1.csv')
cnt_changes_items = create_change(table_item_cnt)
cnt_changes_items.to_csv('data/cnt_changes_items.csv', index=False)
del cnt_changes_items
del table_item_cnt
gc.collect()

"""

"\ntable_item_price = pd.read_csv('data/avg_item_priceitem_id_lag_1.csv')\nprice_changes_items = create_change(table_item_price)\nprice_changes_items.to_csv('data/item_price_change.csv', index=False)\ndel table_item_price\ndel price_changes_items\ngc.collect()\n\nprices = pd.read_csv('data/avg_item_price.csv')\nprice_changes = create_change(prices)\nprice_changes.to_csv('data/shop_item_price_change.csv', index=False)\ndel price_changes\ndel prices\ngc.collect()\n\ncnt_changes_items_shops = create_change(merged)\ncnt_changes_items_shops.to_csv('data/cnt_changes_items_shops.csv', index=False)\ndel cnt_changes_items_shops\ngc.collect()\n\n\ntable_item_cnt = pd.read_csv('data/item_cnt_month_lag_1.csv')\ncnt_changes_items = create_change(table_item_cnt)\ncnt_changes_items.to_csv('data/cnt_changes_items.csv', index=False)\ndel cnt_changes_items\ndel table_item_cnt\ngc.collect()\n\n"

In [16]:
merged=merged.merge(shop_city, on='shop_id', how='left')
merged=merged.merge(category_super_category, on='item_category_id', how='left')
merged['city'] = merged['city'].map(lambda a:a[0])
merged['super_category'] = merged['super_category'].map(lambda a:a[0])

In [19]:
names_base = ['ema_6_item_cnt_month_item_id',
             'ema_6_item_cnt_month_item_id_shop_id',
             'ema_6_item_cnt_month_item_category_id_cat_city',
             'ema_6_item_cnt_month_item_category_id_cat_shop_id',
             'date_block_num_diff',
            'avg_item_priceitem_id_lag_1',
            'item_cnt_monthitem_category_id_cat_lag_1',
            'item_cnt_month_lag_1',
             'avg_item_price']
#Lag in a name left from columns name; Value for curr month stored in validation/data

In [20]:
names_changed = ['item_price_change','shop_item_price_change']

In [21]:
names= names_base + names_changed

In [22]:
len(names)

11

In [23]:
names

['ema_6_item_cnt_month_item_id',
 'ema_6_item_cnt_month_item_id_shop_id',
 'ema_6_item_cnt_month_item_category_id_cat_city',
 'ema_6_item_cnt_month_item_category_id_cat_shop_id',
 'date_block_num_diff',
 'avg_item_priceitem_id_lag_1',
 'item_cnt_monthitem_category_id_cat_lag_1',
 'item_cnt_month_lag_1',
 'avg_item_price',
 'item_price_change',
 'shop_item_price_change']

In [18]:
import pandas as pd

def rename_columns(df, name=None):
    
    new_columns = {}
    for col in df.columns:
        new_col = str(col)
        if new_col in ['shop_id','item_id','item_category_id','super_category','city']:
            continue

        else:
            #print(name)
            #print(new_col)
            new_col = name+'$'+new_col

        
        new_columns[col] = new_col
    
    #print(new_columns)
    df.rename(columns=new_columns, inplace=True)
    
    #return df


In [25]:
critical=['change','mean','ema']

def merge_csvs_boosting(df0, names):
    
    rename_columns(df0, 'shop_item_cnt')
    for df_name in names:
        df = pd.read_csv('data/'+df_name+'.csv')
        print(sum(df.memory_usage()/10**6))

        
        for i in range(FIRST_N_MONTH_TO_DROP):#exclude features where past can not be calculated
            
            if any(cr in df_name for cr in critical):
                
                df=df.drop(str(i), axis=1)
        
        rename_columns(df, df_name)
        
        df0 = pd.merge(df0, df, on=['shop_id','item_id'], how = 'left')
        del df
        gc.collect()

    return df0

names_boosting =['ema_6_item_cnt_month_item_id',
                 'ema_6_item_cnt_month_item_id_shop_id',
                 'ema_6_item_cnt_month_item_category_id_cat_city',
                 'ema_6_item_cnt_month_item_category_id_cat_shop_id',
                 'date_block_num_diff',
                 'avg_item_priceitem_id_lag_1',
                 'item_cnt_monthitem_category_id_cat_lag_1',
                 'item_cnt_month_lag_1',
                 #'avg_item_price',
                 'item_price_change']


merge_csvs_boosting(merged, names_boosting).to_csv('data/merged.csv', index=False)

150.22410000000002


FileNotFoundError: [Errno 2] No such file or directory: 'data/ema_6_item_cnt_month_item_id_shop_id.csv'

In [27]:
merged.columns

Index(['shop_id', 'item_id', 'shop_item_cnt$0', 'shop_item_cnt$1',
       'shop_item_cnt$2', 'shop_item_cnt$3', 'shop_item_cnt$4',
       'shop_item_cnt$5', 'shop_item_cnt$6', 'shop_item_cnt$7',
       'shop_item_cnt$8', 'shop_item_cnt$9', 'shop_item_cnt$10',
       'shop_item_cnt$11', 'shop_item_cnt$12', 'shop_item_cnt$13',
       'shop_item_cnt$14', 'shop_item_cnt$15', 'shop_item_cnt$16',
       'shop_item_cnt$17', 'shop_item_cnt$18', 'shop_item_cnt$19',
       'shop_item_cnt$20', 'shop_item_cnt$21', 'shop_item_cnt$22',
       'shop_item_cnt$23', 'shop_item_cnt$24', 'shop_item_cnt$25',
       'shop_item_cnt$26', 'shop_item_cnt$27', 'shop_item_cnt$28',
       'shop_item_cnt$29', 'shop_item_cnt$30', 'shop_item_cnt$31',
       'shop_item_cnt$32', 'shop_item_cnt$33', 'shop_item_cnt$34',
       'item_category_id', 'city', 'super_category'],
      dtype='object')

In [19]:

def merge_csvs_LSTM(df0, names):
    
    df0=df0.drop('0', axis=1)
    
    rename_columns(df0, 'shop_item_cnt')
    for df_name in names:
        df = pd.read_csv('data/'+df_name+'.csv')
        print(sum(df.memory_usage()/10**6))

        df=df.drop(str(0), axis=1)
    
        rename_columns(df, df_name)
        
        df0 = pd.merge(df0, df, on=['shop_id','item_id'], how = 'left')
        del df
        gc.collect()

    return df0

names_LSTM = ['avg_item_price']


    
merge_csvs_LSTM(merged, names_LSTM).to_csv('data/merged.csv', index=False)

150.22410000000002
